In [2]:
import findspark

# spark location on namenode server
findspark.init("/usr/hdp/current/spark2-client")
import pyspark
conf = pyspark.SparkConf().setAll([('spark.app.name', 'multiple_trips_run'), # App Name
    ('spark.master', 'yarn'),              # spark run mode: locally or remotely
    ('spark.submit.deployMode', 'client'), # deploy in yarn-client or yarn-cluster
    ('spark.executor.memory', '8g'),       # memory allocated for each executor
    ('spark.executor.cores', '3'),         # number of cores for each executor
    ('spark.executor.instances', '1'),    # number of executors in total
    ('spark.driver.maxResultSize', '5g'), # Result size is large, need to increase from default of 1g
    ('spark.yarn.am.memory', '10g')])       # memory for spark driver (application master)
sc = pyspark.SparkContext.getOrCreate(conf=conf)

from pyspark.sql import SparkSession 
spark = SparkSession.builder.getOrCreate()

In [3]:
from datetime import date, timedelta
d1 = date(2017,5,17)
d2 = date(2018,9,10)
delta = d2 - d1
days=[]
for i in range(delta.days+1):
    days.append(d1+timedelta(i))

In [4]:
import subprocess

data_files=[]


def run_cmd(args_list):
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    proc.communicate()
    return proc.returncode

for day in days:
    day = day.isoformat()
    dd = day.replace('-', '')
    df_name='hdfs://172.15.7.170:8020/data/guobiao/csv/d={}'.format(dd)
    returncode = run_cmd(['hdfs', 'dfs', '-test', '-e', df_name])
    if returncode:
        print('{} does not exist, skipping ..'.format(df_name))
        continue    
    data_files.append(df_name)

hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170519 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170520 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170521 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170523 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170524 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170525 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170526 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170527 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170528 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170529 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170530 does not exist, skipping ..
hdfs://172.15.7.170:8020/data/guobiao/csv/d=20170601 does not exist, skipping ..
hdfs://172.15.7.170:8020/dat

In [12]:
#from vehicle_utils import load_header_config

#config_file = '/home/wchen/guobiao/guobiao_header.csv'

if len(data_files) == 1:
    rdd = spark.sparkContext.textFile(data_files[0])
else:
    rdds = []
    for data_file in data_files:
        
        rddi = spark.sparkContext.textFile(data_file)
        rdds.append(rddi)
        rdd = sc.union(rdds)


# Convert to spark df
#spark_df = rdd.map(lambda x: str(x)).map(lambda w: w.split(',')).toDF()
#header_columns, configs = load_header_config(config_file)
#spark_df = spark_df.toDF(*header_columns)

In [14]:
def parseline(line):
    fields=line.split(',')
    vin=fields[0]     #no conversion would be string
    otherFields=[]
    vintype=fields[1]
    timestamp=long(fields[2])
    veh_st=int(fields[3])
    veh_spd=long(fields[6])
    veh_odo=long(fields[7])
    loc_lon=long(fields[28])
    loc_lat=long(fields[29])
    alm_lvl_higest=int(fields[42])
    otherFields.
    return (vin, vintype, timestamp, veh_st, veh_spd, veh_odo, loc_lon,loc_lat, alm_lvl_higest)

In [15]:
data=rdd.map(parseline)

In [17]:
results=data.collect()
for result in results:
    print(results[0])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 12, datanode1, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/hadoop/yarn/local/usercache/wchen/appcache/application_1535492291517_5113/container_e14_1535492291517_5113_01_000003/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/hadoop/yarn/local/usercache/wchen/appcache/application_1535492291517_5113/container_e14_1535492291517_5113_01_000003/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/hadoop/yarn/local/usercache/wchen/appcache/application_1535492291517_5113/container_e14_1535492291517_5113_01_000003/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs =